In [1]:
import ast
import os
import glob
import itertools as it
import copy
import collections as cc
import pandas as pd
import spacy as sp
import torch as tc
import torch.nn
import torch.cuda
import torch.linalg as tcl
import torch.utils.data as tcd
import logging
import numpy as np
import numpy.random as npr
import argparse

import transformers as transf
import azureml.core as aml

from transformers import BertModel, BertConfig, BertForMaskedLM, BertTokenizer, AdamW
from azureml.core import Run
from enum import Flag


In [2]:
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)
logging.basicConfig(filename = "./log/logTagAnalysis.txt", format = '%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt = '%m/%d/%Y %H:%M:%S',
                    level = logging.INFO)

__amlrun = Run.get_context()

In [3]:
class DebugOption(Flag):
    Empty = 0
    General = 1
    ScoreAgg = 2
    Eval = 4

class TrainingConfig:
    def initDevice(self, _useGPU = False):
        logger_ = logging.getLogger(TrainingConfig.__name__)        
        if _useGPU and self.debug == DebugOption.Empty:
            logger_.info("Use GPU Enabled")
            if tc.cuda.is_available():
                self.device = tc.device("cuda")
                logger_.info("GPU initialized")
                return
            else:
                logger_.info("GPU fail to initialize")
        else:
            logger_.info("Use GPU Disabled (or in debugging)")
        logger_.info("Use CPU by default")
        self.device = tc.device("cpu")

    def __init__(self):
        self.learningRate = 0.00003
        self.weightDecay = 0.01
        self.maxSentenceLength = 256
        self.device = tc.device("cpu")
        self.evalEpochs = 1
        self.evalEpochSize = 2500
        self.epochSize = 2500
        self.batchSize = 32
        self.batchAccumulation = 2
        self.epochs = 1
        self.warmUp = 0.2
        self.removeAggBias = 1
        self.maskRate = 0.25
        self.debug = DebugOption.Empty
        self.dataInputPath = "./data/rclean/"
        self.dataModelPath = "./models/"
        
    
    def __str__(self):
        return str(vars(self))

In [4]:
# Global config
__config = TrainingConfig()

## Global States

In [5]:
# Global models
__model = BertForMaskedLM.from_pretrained('bert-base-uncased').to(__config.device)
__tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
__tokenizer.convert_tokens_to_ids("[CLS]")

101

In [7]:
def printToken(_id):
    if isinstance(_id, str):
        _id = __tokenizer.convert_tokens_to_ids(_id)
    print(_id, tc.norm(__model.cls.predictions.decoder.weight[_id]), __model.cls.predictions.decoder.bias[_id])

printToken("[CLS]")
printToken("[MASK]")
printToken("[SEP]")

101 tensor(2.0308, grad_fn=<CopyBackwards>) tensor(-3.5362, grad_fn=<SelectBackward>)
103 tensor(1.5410, grad_fn=<CopyBackwards>) tensor(-4.1730, grad_fn=<SelectBackward>)
102 tensor(0.7657, grad_fn=<CopyBackwards>) tensor(-4.5105, grad_fn=<SelectBackward>)


In [8]:
# Global states
__curEpoch = 0
__extraTokens = dict()

In [34]:
def saveCheckpoint(_name):
    state_ = {"epoch": __curEpoch, "model" : __model.state_dict(), "extratokens" : __extraTokens}

    tc.save(state_, os.path.join(__config.dataModelPath, _name + ".chk"))
    
def loadCheckpoint(_name):
    global __curEpoch
    global __extraTokens

    state_ = tc.load(os.path.join(__config.dataModelPath, _name + ".chk"), map_location = __config.device)
    __model.load_state_dict(state_["model"])
    __curEpoch = state_["epoch"]
    __extraTokens = state_["extratokens"]
    for token_ in __extraTokens.values():
        token_.to(__config.device)

## Data Prep

In [10]:
def toList(_str):
    try:
        return ast.literal_eval(_str.replace('-', ' '))
    except Exception:
        if "nan" in _str:
            return toList(_str.replace("nan", "''"))
        else:
            return []

In [11]:
def subTokens(_target, _candidates):
    res_ = [0] * len(_target)
    for k, c in enumerate(_candidates):
        for i, t in enumerate(_target):
            if i + len(c) > len(_target):
                break
            if res_[i] or t != c[0]:
                continue
            for j, w in enumerate(c):
                if w != _target[i + j] or res_[i + j]:
                    break
            else:
                for j in range(i, i + len(c)):
                    res_[j] = k + 1
    return res_
print(subTokens(['abcd', 'def', 'zzz', 'zzz'], [['abc', 'def'], ['def', 'zzz'], ['zzz'], ['xyz']]))
print(subTokens(['that', 'is', 'the', 'right'], [['is', 'the'], ['thats', 'is'], ['is'], ['the', 'right']]))

[0, 2, 2, 3]
[0, 1, 1, 0]


In [12]:
def generateBias(_vec: tc.tensor):
    with tc.no_grad():
        w = tc.norm(_vec).item()
        b = tc.tensor(-3.0)
        return _vec * 2.0 / w, b

In [13]:
def resetEpoch(_vers = 0, _push = True):
    logger_ = logging.getLogger(resetEpoch.__name__)
    
    global __curEpoch
    __curEpoch = _vers    
    
    logger_.info("Before push extra token count: " + str(len(__extraTokens.values())))
    for sc_ in list(__extraTokens.values()):
        sc_.version = _vers
        if _push:
            if sc_.push(__model.cls.predictions.decoder) == 0:
                __extraTokens[sc_.name].isEval = False
    
    logger_.info("Post push extra token count: " + str(len(__extraTokens.values())))

def syncBatchTokens():
    for sc_ in list(__extraTokens.values()):
        if sc_.isEval:
            sc_.update(__model.cls.predictions.decoder)

In [14]:
class Scores:
    def __init__(self, _words, _embedding, _isEval):        
        self.isEval = _isEval
        self.assign(0, -1)
        self.num = 0
        self.name = _words
        self.tensor = tc.zeros_like(_embedding)
        self.tensor2 = tc.zeros_like(_embedding)
        self.bias = tc.tensor(0.0)
        self.bias2 = tc.tensor(0.0)
    
    def assign(self, _slot, _vers):
        assert(self.isEval)        
        self.id = _slot
        self.version = _vers
    
    def append(self, _tensor : tc.Tensor, _bias : tc.Tensor = tc.tensor(0.0)):
        assert tc.is_nonzero(_tensor[0])
        with tc.no_grad():
            self.tensor2 += _tensor.detach()
            self.bias2 += _bias.detach()
        
        self.num += 1
    
    def get(self):
        return self.tensor, self.bias
    
    def valid(self):  
        return self.isEval and tc.is_nonzero(self.tensor[0])
    
    def update(self, _decoder):
        with tc.no_grad():
            assert tc.is_nonzero(_decoder.weight[self.id][0])
            self.tensor.copy_(_decoder.weight[self.id].detach())
            self.bias.copy_(_decoder.bias[self.id].detach())

    def getId(self, _version):
        return self.id
    
    def push(self, _decoder = None):
        with tc.no_grad():
            self.tensor.copy_(self.tensor2 / self.num if self.num else self.tensor2)
            self.bias.copy_(self.bias2 / self.num if self.num else self.bias2)            
            self.tensor2.zero_()
            self.bias2.zero_()
            
            if self.num > 0 and _decoder:
                _decoder.weight[self.id].copy_(self.tensor.detach())
                _decoder.bias[self.id].copy_(self.bias.detach())
                _decoder.weight[self.id].requires_grad_(True)
                _decoder.bias[self.id].requires_grad_(True)

        r = self.num
        self.num = 0
        return r
    
    def to(self, _device):
        if self.tensor is not None:
            self.tensor.to(_device)
        if self.tensor2 is not None:
            self.tensor2.to(_device)
        if self.bias is not None:
            self.bias.to(_device)
        if self.bias2 is not None:
            self.bias2.to(_device)            

In [15]:
def extractUnusedIds(_vocab, _range = 1000):
    for i in range(_range):
        t = "[unused{0}]".format(i)
        if t in _vocab:
            yield _vocab[t]

def createTokenName(_words):
    return '+'.join(_words) + '&'

def allocScore(_embedding, _slots, _words, _vers, _isEval = False):
    if _words not in __extraTokens:
        __extraTokens[_words] = Scores(_words, _embedding.weight[0], _isEval)
        assert not __extraTokens[_words].id
    
    sc_ = __extraTokens[_words]
    
    id_ = sc_.getId(_vers)
    
    if not id_:
        assert len(_slots) > 1
        id_ = sc_.assign(_slots.pop(), _vers)
    
    __tokenizer.vocab[_words] = id_
    __tokenizer.ids_to_tokens[id_] = _words
    
    return sc_

### Sequence Generation

In [16]:
def generateTokenizedSequence(_texts, _candidates, _slots, _vers, _isEval = False):
    logger_ = logging.getLogger(generateTokenizedSequence.__name__)
    _candidates = [__tokenizer.tokenize(t) for t in _candidates if t]
    _candidates = [c for c in _candidates if allocScore(__model.bert.embeddings.word_embeddings, _slots, createTokenName(c), _vers, _isEval)]
    
    def generateMaskCopy(_tokens, _kpes, _maskToken):        
        ids_ = [i for i in range(1, len(_tokens) - 1) if _tokens[i] != "[SEP]"]
        npr.shuffle(ids_)
        mask_ = copy.copy(_tokens)
        for i in it.islice(ids_, max(1, int(len(ids_) * __config.maskRate))):
            mask_[i] = _maskToken
        return mask_
    
    def expandSentence(_tokens, _kpes, _candidates):
        i = 0
        tokensExpands_ = list()
        kpesExpands_ = list()

        while i < len(_tokens):
            if not _kpes[i]:
                tokensExpands_.append(_tokens[i])
                kpesExpands_.append(0)
                i += 1
                continue

            j = i + 1            
            while j < len(_tokens) and _kpes[j] == _kpes[i]:
                j += 1
            
            if len(tokensExpands_) + len(_tokens) - i + 4 < __config.maxSentenceLength:
                c = _kpes[i]
                
                for z in range(i, j):
                    tokensExpands_.append(_tokens[z])
                    kpesExpands_.append(0)
                
                tokensExpands_.append('(')
                tokensExpands_.append("aka")                
                name_ = createTokenName(_candidates[c - 1])
                if _isEval:
                    tokensExpands_.append('[MASK]')
                else:
                    assert __extraTokens[name_].valid()
                    tokensExpands_.append(name_)
                tokensExpands_.append(')')
                
                kpesExpands_.append(-1)
                kpesExpands_.append(-1)
                kpesExpands_.append(__extraTokens[name_].getId(_vers))                               
                kpesExpands_.append(-1)
                i = j
            else:
                tokensExpands_.extend(_tokens[i:])
                kpesExpands_.extend(it.repeat(0, len(_tokens) - i))
                break
        assert len(tokensExpands_) == len(kpesExpands_)
        return tokensExpands_, kpesExpands_
        
    tokens_ = ["[CLS]"]
    for text_ in _texts.split("\r\n"):
        curTokens_ = __tokenizer.tokenize(text_)
        if len(tokens_) + len(curTokens_) > __config.maxSentenceLength - 20:
            break
        tokens_ += curTokens_ + ["[SEP]"]

    kpes_ = subTokens(tokens_, _candidates)
    
    tokens_, kpes_ = expandSentence(tokens_, kpes_, _candidates)
    
    if _isEval:
        return tokens_, tokens_, kpes_
    else:
        return tokens_, generateMaskCopy(tokens_, kpes_, "[MASK]"), kpes_
    
def generateTrainingData(_data, _slots, _isEval = False):
    logger_ = logging.getLogger(generateTokenizedSequence.__name__)    
    
    slots_ = _slots
    def generateRow(_text, _tags):
        tks_, msks_, kpes_ = generateTokenizedSequence(_text, _tags, slots_, __curEpoch, _isEval)
        tks_ = __tokenizer.convert_tokens_to_ids(tks_)        
        msks_ = __tokenizer.convert_tokens_to_ids(msks_)

        sentLen_ = len(tks_)
        activeMask_ = list(map(lambda x : int(x < sentLen_), range(__config.maxSentenceLength)))
        tks_.extend(it.repeat(0, __config.maxSentenceLength - sentLen_))
        msks_.extend(it.repeat(0, __config.maxSentenceLength - sentLen_))
        kpes_.extend(it.repeat(0, __config.maxSentenceLength - sentLen_))
        return pd.Series([tks_, msks_, activeMask_, kpes_], index = ["tokens", "tokenmasks", "masks", "kpes"])
    
    logger_.info("input raw training data %d", _data.shape[0])
    _data = _data.dropna()
    logger_.info("filtering na training data %d", _data.shape[0])
    
    logger_.info("generating features for questions")
    pd1_ = _data.apply(lambda _row: generateRow(_row.CleanBody, _row.Tag), axis = 1)
    logger_.info("generating features for answers")
    pd2_ = _data.apply(lambda _row: generateRow(_row.AnswerBody, _row.Tag), axis = 1)
    logger_.info("completed generating features")
    
    return pd.concat([pd1_, pd2_])


def convertToTensor(_data):
    #tmasks_ = [list(map(lambda x : 1 if x != 0 else 0, kpe_)) for kpe_ in _data.kpes]
    masks_ = tc.tensor(_data.masks.array, dtype = tc.long, device = __config.device)
    #tmasks_ = tc.tensor(tmasks_, dtype = tc.long, device = __config.device)
    tmasks_ = tc.zeros_like(masks_, dtype = tc.long, device = __config.device)
    return tcd.TensorDataset(
        tc.tensor(_data.tokenmasks.array, dtype = tc.long, device = __config.device),
        tmasks_,
        masks_,
        tc.tensor(_data.tokens.array, dtype = tc.long, device = __config.device))
  

## Training

In [17]:
def createOptimizer(_lrFact = 1):
    FilterNodecay_ = lambda x : any(map(lambda t : t in x, ["bias", "LayerNorm.weight"]))
    parameters_ = list(__model.named_parameters())
    parameters_ = [
        {"params": [p for n, p in parameters_ if not FilterNodecay_(n)], 'weight_decay': __config.weightDecay},
        {"params": [p for n, p in parameters_ if FilterNodecay_(n)], 'weight_decay': 0.0}
    ]
    return AdamW(parameters_, lr = __config.learningRate * _lrFact, correct_bias=False)  # To reproduce BertAdam specific behavior set correct_bias=False

In [18]:
def trainBert(_dataFrame, _optimizer):
    logger_ = logging.getLogger(trainBert.__name__)
    __model.train()
    
    logger_.info("training data frame size %d", _dataFrame.shape[0])
    
    _trainingDataset = convertToTensor(_dataFrame)
    
    trainingSteps_ = len(_trainingDataset) // (__config.batchSize * __config.batchAccumulation)
    
    logger_.info("begin {0} epoch training, with {1} steps on device {2} ...".format(__curEpoch, trainingSteps_, __config.device))
    
    dataReader_ = tcd.DataLoader(_trainingDataset, shuffle = True, batch_size = __config.batchSize, drop_last = True)
    _optimizer.zero_grad()
    for step_, batch_ in enumerate(dataReader_):
        output_ = __model(input_ids = batch_[0], attention_mask = batch_[2], token_type_ids = batch_[1], labels = batch_[3])
        #loss_ = predCost(output_[0], batch_[3], batch_[1]) / __config.batchAccumulation
        loss_ = output_[0] / __config.batchAccumulation

        loss_.backward()
        
        if not ((step_ + 1) % __config.batchAccumulation):
            # tc.nn.utils.clip_grad_norm_(__model.parameters(), 1.0)
            _optimizer.step()
            _optimizer.zero_grad()            
            __amlrun.log('loss', loss_)
            logger_.info("step {0}/{1} finished, loss : {2}".format((step_ + 1) // __config.batchAccumulation, trainingSteps_, loss_))
    
    logger_.info("end {0} epoch training".format(__curEpoch))

In [19]:
def printTokens(_tokens):
    print(_tokens)
    print(__tokenizer.convert_ids_to_tokens(_tokens.numpy()))

def printEmbedding(_token):
    print(__model.bert.embeddings.word_embeddings.weight[_token])
    print(__model.bert.embeddings.word_embeddings.weight.grad[_token])

In [20]:
def getEmbeddings2(_tokens, _tokenTypes, _masks, _labels = None):
    logger_ = logging.getLogger(getEmbeddings.__name__)
    
    with tc.no_grad():
        outputs_ = __model.bert(
            _tokens,
            attention_mask=_masks,
            token_type_ids=_tokenTypes)
        output_ = __model.cls.predictions.transform(outputs_[0])

        indexes_ = __model.cls.predictions.decoder(output_)
        index_ = tc.argmax(indexes_, dim = 2)
        return output_, index_

def getEmbeddings(_tokens, _tokenTypes, _masks, _labels = None):
    logger_ = logging.getLogger(getEmbeddings.__name__)
    
    with tc.no_grad():
        output_ = __model(input_ids = _tokens, token_type_ids = _tokenTypes, attention_mask = _masks)
        normalizer_ = tc.nn.Softmax(2)
        output_ = normalizer_(output_[0])
        output_[:, :, : 1000].zero_()
        
        index_ = tc.argmax(output_, dim = 2)
        
        if __config.removeAggBias:
            bias_ = tc.norm(__model.bert.embeddings.word_embeddings.weight, p = 2, dim = 1, keepdim = True) + 1e-12
            bases_ = tc.div(__model.bert.embeddings.word_embeddings.weight, bias_)
            bias_ = tc.mean(bias_)
            output_ = tc.matmul(output_, bases_) # * bias_
        else:
            
            output_ = tc.matmul(output_, __model.bert.embeddings.word_embeddings.weight)
            
        return output_, index_

def evaluateModel(_dataFrame):
    logger_ = logging.getLogger(evaluateModel.__name__)

    __model.eval()
    evaluationSet_ = convertToTensor(_dataFrame)
    evalSteps_ = len(evaluationSet_) // __config.batchSize

    dataReader_ = tcd.DataLoader(evaluationSet_, shuffle = False, batch_size = __config.batchSize, drop_last = False)
    
    debug_ = 0
    for step_, batch_ in enumerate(dataReader_):
        logger_.info("get gradient for steps %d/%d", step_, evalSteps_)
        embeddings_, ptk_ = getEmbeddings2(*batch_)
        batchSize_ = embeddings_.shape[0]
        debug_ += batchSize_
        for i in range(batchSize_):
            recorder_ = _dataFrame.iloc[step_ * __config.batchSize + i]
            if not any(recorder_.kpes):
                continue
            assert len(recorder_.kpes) == len(embeddings_[i])
            for k, e, t in zip(recorder_.kpes, embeddings_[i], ptk_[i]):
                if k > 0:
                    __extraTokens[__tokenizer.convert_ids_to_tokens(k)].append(*generateBias(e))
    assert debug_ == len(_dataFrame)
    logger_.info("grandients update completed")

## Evaluate and Update

In [21]:
def readDataFrame(_readChunkSize, _readCounts, _skipFiles = 0):
    counter_ = 0
    for file_ in glob.glob(os.path.join(__config.dataInputPath, "*.csv")):
        if _skipFiles > 0:
            _skipFiles -= 1
            continue

        data_ = pd.read_csv(file_)
        data_.Tag = data_["Tag"].map(toList)
        for i in range(0, data_.shape[0], _readChunkSize):
            counter_ += 1
            if counter_ > _readCounts:
                return
            yield data_.iloc[i : i + _readChunkSize]

In [22]:
def eval1(_startPoint = 0):
    logger_ = logging.getLogger(eval1.__name__)
    global __curEpoch
 
    logger_.info("Begin aggregation process")
    __extraTokens.clear()
    slots_ = list(extractUnusedIds(__tokenizer.vocab, 1000))

    resetEpoch(0, False)
    for dataFrame_ in readDataFrame(__config.evalEpochSize, __config.evalEpochs, _startPoint):        
        __curEpoch += 1
        dataFrame_ = generateTrainingData(dataFrame_, slots_, True)
        evaluateModel(dataFrame_)    
    
    resetEpoch()

In [23]:
def trainAll(_startPoint = 0):
    logger_ = logging.getLogger(trainAll.__name__)
    
    global __curEpoch
 
    logger_.info("Begin aggregation process")
    slots_ = list(extractUnusedIds(__tokenizer.vocab, 1000))

    resetEpoch(0, False)
    for dataFrame_ in readDataFrame(__config.evalEpochSize, __config.evalEpochs, _startPoint):        
        __curEpoch += 1
        dataFrame_ = generateTrainingData(dataFrame_, slots_, True)
        evaluateModel(dataFrame_)    

    logger_.info("Begin training process")
    
    resetEpoch()
    testTopics()
    testTopics3()
    optimizer_ = createOptimizer()
    backups_ = backupTopics()
    for dataFrame_ in readDataFrame(__config.epochSize, __config.epochs, _startPoint):
        __curEpoch += 1
        dataFrame_ = generateTrainingData(dataFrame_, slots_)
        trainBert(dataFrame_, optimizer_)
    
    syncBatchTokens()
    testTopics()
    testTopics3()    
    logger_.info("training completed aggregation process")    

In [24]:
def backupTopics():
    backup_ = dict()
    for t, v in __extraTokens.items():
        backup_[t] = v.tensor.detach().clone()
    return backup_

## Evaluation

In [25]:
def extractWordEmbeddings():
    tensors_ = list()
    for name_ in topKTopic._names:
        if name_ == "c++++&":
            tokens_ = ['c', '+', '+']
        else:
            tokens_ = name_.strip('&').split('+')
        tokens_ = __tokenizer.convert_tokens_to_ids(tokens_)
        embeddings_ = list(map(lambda x : __model.bert.embeddings.word_embeddings.weight[x], tokens_))
        embedding_ = sum(embeddings_) / len(embeddings_)
        
        tensors_.append(embedding_)
    
    tensors_ = tc.stack(tensors_, dim = 0)    
    tensors_ = tensors_ / (tc.norm(tensors_, p = 2, dim = 1, keepdim = True) + 1e-12)
    
    return tensors_


In [26]:
tc.norm(tc.sub(tc.tensor([[1, 2], [3, 4]], dtype = tc.float), tc.tensor([2, 3], dtype = tc.float)), dim = 1)

tensor([1.4142, 1.4142])

In [27]:
def extractTokens():
    goodTokens_ = list(filter(lambda x : x.isEval, __extraTokens.values()))
    names_ = list(map(lambda x : x.name, goodTokens_))
    tensors_ = tc.stack(list(map(lambda x : x.tensor.detach(), goodTokens_)), dim = 0)
    tensors_ = tensors_ / (tc.norm(tensors_, p = 2, dim = 1, keepdim = True) + 1e-12)
    return names_, tensors_

def extractTokens2():
    goodTokens_ = list(filter(lambda x : x.isEval, __extraTokens.values()))
    names_ = list(map(lambda x : x.name, goodTokens_))
    tensors_ = tc.stack(list(map(lambda x : x.tensor.detach(), goodTokens_)), dim = 0)
    #tensors_ = tensors_ / (tc.norm(tensors_, p = 2, dim = 1, keepdim = True) + 1e-12)
    return names_, tensors_

def topKTopic2(t, _top = 5):
    if isinstance(t, str):
        t = __extraTokens[t].tensor
    
    distances_ = tc.norm(tc.sub(topKTopic2._tensors, t), dim = 1)
    _, ids_ = tc.topk(distances_, _top, dim = 0, largest = False)
    
    return list(map(lambda x : topKTopic2._names[x], ids_))

def topKTopic(t, _top = 5):    
    if isinstance(t, str):
        t = __extraTokens[t].tensor
    _, ids_ = tc.topk(tc.matmul(topKTopic._tensors, t), _top, dim = 0)
    
    return list(map(lambda x : topKTopic._names[x], ids_))

def distance(_left, _right):
    print(tc.dot(__extraTokens[_left].tensor, __extraTokens[_right].tensor))

# topKTopic("flex&")

In [28]:
def testTopic(_topic):
    print("Topic '{0}' related to {1}".format(_topic, str(topKTopic(__extraTokens[_topic].tensor))))

def testTopic2(_topic):
    print("Topic '{0}' related to {1}".format(_topic, str(topKTopic2(__extraTokens[_topic].tensor))))

def testTopics2():
    topKTopic._names, topKTopic._tensors = extractTokens()
    topKTopic._tensors = extractWordEmbeddings()

    for i, name_ in enumerate(topKTopic._names):
        if len(tc.nonzero(topKTopic._tensors[i])):
            print("Topic '{0}' related to {1}".format(name_, str(topKTopic(topKTopic._tensors[i]))))
    
def testTopics():
    topKTopic._names, topKTopic._tensors = extractTokens()
    for i, name_ in enumerate(topKTopic._names):
        assert(tc.is_nonzero(topKTopic._tensors[i, 0]))
        testTopic(name_)
    print("-------------------------")
        
def testTopics3():
    topKTopic2._names, topKTopic2._tensors = extractTokens2()
    for i, name_ in enumerate(topKTopic2._names):
        assert(tc.is_nonzero(topKTopic2._tensors[i, 0]))
        testTopic2(name_)
    print("-------------------------")

## Main

In [29]:
def main(_inputDataPath = None, _outputModelPath = None, _Epochs = 4, _useGPU = False, _debug = DebugOption.Empty):
    logger_ = logging.getLogger(main.__name__)
    __config.initDevice(_useGPU)

    __config.debug = _debug
    
    if _Epochs > 0:
        __config.epochs = _Epochs
        __config.evalEpochs = _Epochs    
    
    if _inputDataPath:
        __config.dataInputPath = _inputDataPath
    if _outputModelPath:
        __config.dataModelPath = _outputModelPath
    
    logger_.info("Configs: " + str(__config))
    
    __extraTokens.clear()
    trainAll()
    saveCheckpoint("base1")
    
    logger_.info("Training completed, begin eval...")    
    eval1()
    testTopics()
    logger_.info("Completed")

In [31]:
testTopics()

Topic 'sv+##n&' related to ['sv+##n&', 'd+##ns&', 'j+##son&', 'cm+##d&', 'ss+##l&']
Topic 'sql&' related to ['sql&', 'lin+##q+to+sql&', 'sql+##ite&', 'database&', 'sql+server&']
Topic 'as+##p+.+net&' related to ['as+##p+.+net&', '.+net&', 'v+##b+.+net&', 'web&', 'net+##be+##ans&']
Topic 'algorithm&' related to ['algorithm&', 'encoding&', 'if+##ram+##e&', 'reference&', 'routing&']
Topic 'colors&' related to ['colors&', 'types&', 'pointer+##s&', 'font+##s&', 'plug+##ins&']
Topic 'c+#&' related to ['c+#&', 'f+#&', 'c&', 'c++++&', 'c+#+4+.+0&']
Topic '.+net&' related to ['.+net&', 'as+##p+.+net&', 'v+##b+.+net&', 'web&', 'net+##be+##ans&']
Topic 'c++++&' related to ['c++++&', 'visual+c++++&', 'c&', 'c+#&', 'objective+c&']
Topic 'class&' related to ['class&', '.+h+##ta+##cc+##ess&', 'se+##d&', 'if+##ram+##e&', 'cm+##d&']
Topic 'web+services&' related to ['web+services&', 'reporting+services&', 'service&', 'web+applications&', 'generic+##s&']
Topic 'sql+server&' related to ['sql+server&', 'd

In [36]:
testTopics()

Topic 'sv+##n&' related to ['sv+##n&', 'gi+##t&', 'ft+##p&', 'wc+##f&', 'cm+##d&']
Topic 'sql&' related to ['sql&', 'lin+##q+to+sql&', 'ts+##q+##l&', 'sql+##ite&', 'my+##s+##q+##l&']
Topic 'as+##p+.+net&' related to ['as+##p+.+net&', '.+net&', 'share+##point&', 'as+##p+.+net+mv+##c&', 'j+##bos+##s&']
Topic 'algorithm&' related to ['algorithm&', 'rec+##urs+##ion&', 'or+##m&', 'optimization&', 'graph&']
Topic 'colors&' related to ['colors&', 'background&', 'canvas&', 'image&', 'plot&']
Topic 'c+#&' related to ['c+#&', 'c++++&', 'c&', 'f+#&', 'per+##l&']
Topic '.+net&' related to ['.+net&', 'as+##p+.+net&', 'v+##b+.+net&', 'net+##be+##ans&', 'as+##p+.+net+mv+##c&']
Topic 'c++++&' related to ['c++++&', 'c+#&', 'c&', 'st+##l&', 'objective+c&']
Topic 'class&' related to ['class&', 'object&', 'construct+##or&', 'module&', 'function&']
Topic 'web+services&' related to ['web+services&', 'reporting+services&', 'web+applications&', 'service&', 'ss+##is&']
Topic 'sql+server&' related to ['sql+serv

In [35]:
loadCheckpoint("base1")

In [ ]:
if __name__ == "__main__":
    main()

Topic 'sv+##n&' related to ['sv+##n&', 'gi+##t&', 'wc+##f&', 'ft+##p&', 'ss+##l&']
Topic 'sql&' related to ['sql&', 'lin+##q+to+sql&', 'ts+##q+##l&', 'sql+##ite&', 'my+##s+##q+##l&']
Topic 'as+##p+.+net&' related to ['as+##p+.+net&', '.+net&', 'as+##p+.+net+mv+##c&', 'share+##point&', 'v+##b+.+net&']
Topic 'algorithm&' related to ['algorithm&', 'rec+##urs+##ion&', 'or+##m&', 'graph&', 'optimization&']
Topic 'colors&' related to ['colors&', 'background&', 'canvas&', 'image&', 'tab+##s&']
Topic 'c+#&' related to ['c+#&', 'c++++&', 'c&', 'f+#&', 'per+##l&']
Topic '.+net&' related to ['.+net&', 'as+##p+.+net&', 'v+##b+.+net&', 'net+##be+##ans&', 'as+##p+.+net+mv+##c&']
Topic 'c++++&' related to ['c++++&', 'c+#&', 'c&', 'st+##l&', 'objective+c&']
Topic 'class&' related to ['class&', 'object&', 'construct+##or&', 'module&', 'function&']
Topic 'web+services&' related to ['web+services&', 'reporting+services&', 'web+applications&', 'service&', 'ss+##is&']
Topic 'sql+server&' related to ['sql+s

Topic 'script+##ing&' related to ['script+##ing&', 'java+##script&', 'logging&', 'serial+##ization&', 'par+##sing&']
Topic 'rest&' related to ['rest&', 'wc+##f&', 'hi+##ber+##nate&', 'ca+##ching&', 'j+##sp&']
Topic 'soap&' related to ['soap&', 'wc+##f&', 'ss+##l&', 'ser+##v+##lets&', 'j+##son&']
Topic 'lin+##q+to+sql&' related to ['lin+##q+to+sql&', 'sql&', 'ts+##q+##l&', 'sql+##ite&', 'lin+##q&']
Topic 'w+##pf&' related to ['w+##pf&', 'mv+##v+##m&', 'm+##fc&', 'x+##am+##l&', 'j+##que+##ry&']
Topic 'oracle&' related to ['oracle&', 'sql+##ite&', 'jp+##a&', 'or+##m&', 'jd+##bc&']
Topic 'http&' related to ['http&', 'wc+##f&', 'ft+##p&', 'proxy&', 'ss+##l&']
Topic 'de+##bu+##gging&' related to ['de+##bu+##gging&', 'ca+##ching&', 'logging&', 'par+##sing&', 'build&']
Topic 'x+##sl+##t&' related to ['x+##sl+##t&', 'xp+##ath&', 'j+##son&', 'j+##sp&', 'x+##am+##l&']
Topic 'image&' related to ['image&', 'hyper+##link&', 'view&', 'if+##ram+##e&', 'text&']
Topic 'process&' related to ['process&', 

Topic 'ui+##we+##b+##view&' related to ['ui+##we+##b+##view&', 'ui+##table+##view&', 'data+##gr+##id+##view&', 'if+##ram+##e&', 'w+##pf&']
Topic 'compilation&' related to ['compilation&', 'serial+##ization&', 'binding&', 'par+##sing&', 'data+binding&']
Topic 'graph&' related to ['graph&', 'tree&', 'data+##table&', 'grid+##view&', 'data+##gr+##id+##view&']
Topic 'mage+##nto&' related to ['mage+##nto&', 'pay+##pal&', 'facebook&', 'amazon+s+##3&', 'tom+##cat&']
Topic 'ui+##view&' related to ['ui+##view&', 'ui+##we+##b+##view&', 'ui+##table+##view&', 'ui+##sc+##roll+##view&', 'w+##pf&']
Topic 'visual+studio+2010&' related to ['visual+studio+2010&', 'c+#+4+.+0&', 'visual+studio+2008&', 'visual+studio&', 'silver+##light&']
Topic 'cocoa+touch&' related to ['cocoa+touch&', 'active+##re+##cor+##d&', 'm+##fc&', 'as+##p+.+net+mv+##c&', 'q+##t&']
Topic 'character+encoding&' related to ['character+encoding&', 'encoding&', 'cs+##s&', 'x+##sl+##t&', 'par+##sing&']
Topic 'mat+##pl+##ot+##lib&' related